# Thermal Storage Systems
<br/><br/>
Thermal storage systems are systems that are designed to store thermal energy for later use. They are used to capture excess thermal energy during periods of low demand, such as during the night, and then release the energy during periods of high demand, such as during the day. Thermal storage systems can take a variety of forms, including:
<br/><br/>
* Water-based systems: These systems store thermal energy in large tanks of water, which can be heated or cooled as required.

* Ice-based systems: These systems use excess energy to freeze water, which is then stored until it is needed. When the stored ice is melted, the resulting chilled water can be used to cool buildings or industrial processes.

* Phase-change materials (PCMs): These materials store thermal energy by changing phase (e.g. from a solid to a liquid) as the temperature changes. PCMs can be incorporated into building materials, such as walls or ceilings, to store and release thermal energy.

* Molten salt systems: These systems store thermal energy in a molten salt solution, which can be heated or cooled as required.
<br/><br/>

Thermal storage systems are used in a variety of applications, including district heating and cooling, industrial processes, and renewable energy systems such as solar and wind power. They help to improve energy efficiency, reduce peak demand, and provide backup energy when needed.
<br/><br/>
### Steps for thermal storage:
<br/><br/>
* Charge
* Store
* Discharge
<br/><br/>

### Three types of heat storage:
<br/><br/>
* Sensible
* Latent
* Thermochemical
<br/><br/>


# Design of Thermal Storage
<br></br>
The following design procedure will establish specifications for how to properly size thermal storage systems needed in a variety of situations. For the simplicity of the user the input parameters which help properly model the system will be provided by him/her in the very beginning - Section 1: Input parameters. The functions and formulas will be provided in the second portion - Section 2: Formulas and Functions. Section 3: Results - will be focusing on the implementation of the parameters, design conditions, etc. implemented in the previous sections. In Section 4: Design insight - the user can find any additional considerations, assumptions made, limitations of the design procedure, reference models and studies.
<br/><br/>
Section 1: Input parameters
<br/><br/>
Section 2: Formulas and Functions
<br/><br/>
Section 3: Results
<br/><br/>
Section 4: Design insight
<br/><br/>

In comparison to previous design procedures the following will attempt to undergo a slightly different approach to the implementation of the algorithms and functions for the production of results. All previous WASTE HEAT AND POWER design procedures implemented functional programming, while this will focus on OOP approach - the user will be able to apply the class of the specific thermal storage system of interest and then model their specific case using the assigned object. This will not only streamline the design process, but also help the user add additional functionality if needed.
<br/><br/>

In [1]:
from IPython.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [2]:
# function for sensible heat storage - energy stored in hot water tank E [J], [1J = 1/3.6e6 kWh]

def return_energy_stored_hot_water_tank(V, PwCw, dT):
    return V * PwCw * dT # [J]

In [33]:
from abc import ABC, abstractmethod
from math import pi

import math
import numpy as np
import plotly as pl
import plotly.graph_objects as go
import seaborn
import pandas
import matplotlib.pyplot as plt

# (&nbsp;  )

class ATES:
    
    def __init__(self, m, dynamic_viscosity, pf, k, b, L, D, Q, Q_pumped, cw, ca, H, T_Transmitivity, r, R, PmCm, PwCw, n=0.15):
        super().__init__()
        self.m = m
        self.dynamic_viscosity = dynamic_viscosity
        self.pf = pf
        self.k = k
        self.b = b
        self.L = L
        self.D = D
        self.Q = Q
        self.T_Transmitivity = T_Transmitivity
        self.r = r
        self.R = R
        self.n = n
        self.PmCm = PmCm
        self.PwCw = PwCw
        self.Q_pumped = Q_pumped
        self.cw = cw
        self.ca = ca
        self.H = H
        self.Retardation_factor = self.get_thermal_propagation_retardation_factor_R
        self.r_th = self.get_thermal_radius_r_th
        
    # Darcy's Equation for the pressure at an injection well for Aquifer TES, where m is the mass flow rate into the well and out of the production well, pressure is also a function of the dynamic
    # viscosity, pf - the density of the fluid, the permeability k, the reservoir thichness b, the well spacing L and the well diameter D. This assumes flat, homogeneous, isotropic reservois with
    # a uniform initial hydraulic head, perfect aquicludes above and below, a fully penetrating well, negligible changes in fluid density and system conditions that approach steady state
    # Analogous to the Thiem equation
    
    def get_pressure_at_injection_well_darcy_equation_P_inj(self):
        return self.m * self.dynamic_viscosity * math.log(L/D) / (2 * math.pi * self.pf * self.k * self.b)
    
    def get_drawdown_s(self):
        return self.Q * math.log(self.R / self.r) / (2 * math.pi * self.T_Transmitivity)
    
    # Thermal propagation as defined by the Retardation factor - R - considers the significant impact from injection of heat or the removal of heat (coolth) is the advection of the thermal
    # plume from the wellbore by the natural hydraulic gradient. Heat is transported less than that of the natural groundwater velocity. Heat transport equation, where the ratio of volumetric
    # heat capactiy of porous medium and the fluid phase (in this case water) is calculated.
    
    def get_thermal_propagation_retardation_factor_R(self):
        return self.PmCm / (n * self.PwCw) 
    
    
    # Thermally balanced system well calculation, where well should be roughly 3 times the thermal radius 
    
    def get_thermal_radius_r_th(self):
        return math.sqrt(self.cw * self.Q_pumped / (self.ca * self.H * math.pi))
    
    

class BTES:
    
    def __init__(self, H_max, pc_m, dT):
        self.H_max = H_max # [J] Maximum heat load for the subsurface
        self.pc_m = pc_m # [J / (m3 * K)] Volumetric Heat Capacity for the saturated rock/soil
        self.dT = dT # [degrees C] Change of temperature in the subsurface
        self.r = self.get_radius_r() # radius of the giver array 
        self.V_max = self.get_volume_V_max()
    
    # This function returns the maximum Volume at regular borehole array
    
    def get_volume_V_max(self):
        return self.H_max / (self.pc_m * self.dT)
    
    # Usually use cylindrical arrays due to there improved efficiency (Skarphagen et al., 2019). We can therefore find the radius of the given array using the volume previously calculated.
    # Note depth h = 2r. You can also find the number of boreholes by assuming a typical specific heat extraction in W/m and the maximum heat rate of the system in kW.
    
    def get_cylindrical_volume_V_cyl(self):
        return math.pi * 2 * (self.r ** 3)
    
    def get_radius_r(self):
        return math.pow((self.V_max / (2 * math.pi)), 1/3)
        
    
    
    
    
        

In [29]:
m = 12 # [kg/s] Mass flow rate
dynamic_viscosity =  0.0010016 # [kg/ (m * s)] == [N/(m2) * s] == [Pa * s] The dynamic viscocity of the fluid
pf = 997 # [kg/m³] Density of the fluid
k = 0.15 # [m2] Permeability
b = 0.3 # [m] Reservoir thickness
L = 4 # [m] Well spacing
D = 0.5 # [m] Diameter of the well
Q = 2 # [m3/s] the Volumetric flow rate
Q_pumped = 3000 # [m3] Amount of water (Volume) reinjected/pumped per season
T_Transmitivity = 0.1 # [m2/s], T = kb, but we input bothu
r = 0.15 # [m] the radius of the wellbore
R = 20 # [m] is the distance to the other well
PmCm = 2.8e6 # [J / (m ** 3 * K)] Heat capacity of the Aquifer
PwCw = 4.2e6 # [J / (m ** 3 * K)] Heat capacity of Water
cw = PwCw 
ca = PmCm
H = 20 # [m] Length of the screens
n = 0.15

ATES_1 = ATES(m, dynamic_viscosity, pf, k, b, L, D, Q, Q_pumped, cw, ca, H, T_Transmitivity, r, R, PmCm, PwCw, n)


AttributeError: 'ATES' object has no attribute 'PmCm'

In [30]:
print(ATES_1.get_drawdown_s())

15.57443245498099


In [15]:
print(ATES_1.get_thermal_propagation_retardation_factor_R())

4.444444444444445


In [16]:
print(ATES_1.get_thermal_radius_r_th())

8.462843753216344


In [27]:
H_max = 902e9 # [J], note that 1 GJ = e9 J, The maximum heat load for the subsurface
pc_m = 2.1e6 # [J/ (m3 * K)] The volumetric heat capacity of the system
dT = 11 # [degrees C] The change of temperature of the subsurface

BTES_2 = BTES( H_max, pc_m, dT=)

SyntaxError: invalid syntax (2789762592.py, line 5)